 # Подготовка окружения

In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od
import pandas as pd
import time
from functools import reduce
from pathlib import Path

Скачиваем датасет

In [4]:
dataset_path = Path('imdb-user-reviews', 'song_lyrics.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: absolutezlo
Your Kaggle Key: ··········


100%|██████████| 15.1M/15.1M [00:00<00:00, 75.7MB/s]


# Домашняя работа

Сделайте mapper и reducer, чтобы посчитать среднее и дисперсию оценок за фильм. Реализация через цикл (предпогаем, что мы не знаем сколько у нас фильмов):

In [18]:
import json


n, mean, M2 = 0, 0.0, 0
for path in Path('imdb-user-reviews').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)

print(mean, (M2 / n) ** (1/2))

8.03 1.0517128885774862


На основе этого кода соберите mapper и reducer:

In [19]:
def mapper(path):
    # Ваш код
    return (score, )


def reducer(score_data1, score_data2):
    #  Ваш код
    return n, mean, M2

In [20]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.03 1.0517128885774862
CPU times: user 2.34 ms, sys: 0 ns, total: 2.34 ms
Wall time: 4.6 ms


## Решение:

In [21]:
def mapper(path):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        return (float(info['movieIMDbRating']), )


def reducer(score_data1, score_data2):
    if score_data1 is None and score_data2 is None:
        return None
    elif score_data1 is None:
        return score_data2
    elif score_data2 is None:
        return score_data1
    else:
        scores = []
        if len(score_data1) == 1:
            n, mean, M2 = 0, 0.0, 0
            scores.append(score_data1[0])
        else:
            n, mean, M2 = score_data1
        scores.append(score_data2[0])
        for score in scores:
            n += 1
            delta = score - mean
            mean += delta / n
            M2 += delta * (score - mean)
        return n, mean, M2

In [22]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.03 1.0517128885774862
CPU times: user 2.63 ms, sys: 995 µs, total: 3.62 ms
Wall time: 3.63 ms
